# 🔬 Optimisation Multi-Stratégies : Envelope Parameters

Ce notebook optimise les **paramètres de trading** de la stratégie multi-envelope :
- `ma_base_window` : Réactivité du signal
- `envelopes` : Distance d'entrée
- `size` : Risque par trade  
- `stop_loss` : Protection

**Méthodologie** : Walk-Forward Optimization avec Expanding Window pour éviter l'overfitting.

**⚠️ Important** : L'optimisation de la détection de régime se fait dans `multi_envelope_adaptive.ipynb`.

## 1️⃣ Configuration et chargement des données

In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm  # Changé de tqdm.notebook → tqdm.auto

# Backtest engine
from utilities.strategies.envelopeMulti_v2 import EnvelopeMulti_v2
from utilities.data_manager import ExchangeDataManager

# Système adaptatif
from core import calculate_regime_series, DEFAULT_PARAMS
from core.params_adapter import FixedParamsAdapter, RegimeBasedAdapter
from core.backtest_comparator import BacktestComparator

# Config plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports réussis")

c:\Users\jacks\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports réussis


In [6]:
# ======================
# CONFIGURATION GLOBALE
# ======================

BACKTEST_LEVERAGE = 10
INITIAL_WALLET = 1000
EXCHANGE = "binance"

# Périodes (Expanding Window)
PERIODS = {
    "train_full": {"start": "2022-01-01", "end": "2024-06-30"},  # Optimisation
    "holdout": {"start": "2024-07-01", "end": "2024-12-31"},     # Validation finale (intouchable)
}

# Walk-Forward Folds (Expanding Window)
WF_FOLDS = [
    {"train_start": "2022-01-01", "train_end": "2022-12-31", "test_start": "2023-01-01", "test_end": "2023-06-30", "name": "Fold1_Bear→Recovery"},
    {"train_start": "2022-01-01", "train_end": "2023-06-30", "test_start": "2023-07-01", "test_end": "2023-12-31", "name": "Fold2_Bear+Recovery→Bull"},
    {"train_start": "2022-01-01", "train_end": "2023-12-31", "test_start": "2024-01-01", "test_end": "2024-06-30", "name": "Fold3_Full→Bull"},
]

# Paires représentatives (5 paires pour rapidité)
PAIRS = [
    "BTC/USDT:USDT",
    "ETH/USDT:USDT",
    "SOL/USDT:USDT",
    "BNB/USDT:USDT",
    "AVAX/USDT:USDT",
]

# Paramètres de backtest communs
BACKTEST_PARAMS = {
    "initial_wallet": INITIAL_WALLET,
    "leverage": BACKTEST_LEVERAGE,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006,
    "reinvest": True,
    "liquidation": True,
    "risk_mode": "scaling",
}

print(f"✅ Configuration chargée")
print(f"   Paires: {len(PAIRS)}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
print(f"   Hold-out: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")

✅ Configuration chargée
   Paires: 5
   Walk-Forward Folds: 3
   Hold-out: 2024-07-01 → 2024-12-31


In [ ]:
# Chargement des données
exchange = ExchangeDataManager(
    exchange_name=EXCHANGE,
    path_download="../database/exchanges"  # Pointe vers Backtest-Tools-V2/database/exchanges
)

# Charger TOUTES les données nécessaires (2022-2024)
start_date = "2022-01-01"
end_date = "2024-12-31"

df_list_full = {}
print("📥 Chargement des données...")
for pair in tqdm(PAIRS, desc="Paires"):
    df = exchange.load_data(pair, "1h", start_date=start_date, end_date=end_date)
    df_list_full[pair] = df

# BTC pour détection de régime
df_btc_full = exchange.load_data("BTC/USDT:USDT", "1h", start_date=start_date, end_date=end_date)

oldest_pair = min(df_list_full, key=lambda p: df_list_full[p].index.min())

print(f"\n✅ Données chargées")
print(f"   Période: {start_date} → {end_date}")
print(f"   Paire la plus ancienne: {oldest_pair}")
print(f"   Nombre de barres: {len(df_list_full[oldest_pair])}")

## 2️⃣ Comparaison manuelle de configurations pré-définies

Teste 5 configurations fixes pour comprendre l'impact des paramètres.

In [ ]:
# Configurations pré-définies
MANUAL_CONFIGS = {
    "Conservative": {
        "ma_base_window": 10,
        "envelopes": [0.05, 0.08, 0.12],
        "size": 0.06,
        "stop_loss": 0.20,
    },
    "Standard (Live actuel)": {
        "ma_base_window": 7,
        "envelopes": [0.07, 0.10, 0.15],
        "size": 0.10,
        "stop_loss": 0.25,
    },
    "Aggressive": {
        "ma_base_window": 5,
        "envelopes": [0.09, 0.13, 0.18],
        "size": 0.12,
        "stop_loss": 0.30,
    },
    "Wide Envelopes": {
        "ma_base_window": 7,
        "envelopes": [0.10, 0.15, 0.20],
        "size": 0.08,
        "stop_loss": 0.25,
    },
    "Tight Envelopes": {
        "ma_base_window": 7,
        "envelopes": [0.05, 0.07, 0.10],
        "size": 0.08,
        "stop_loss": 0.25,
    },
}

print(f"📋 {len(MANUAL_CONFIGS)} configurations manuelles définies")
for name, cfg in MANUAL_CONFIGS.items():
    print(f"   - {name}: MA={cfg['ma_base_window']}, Env={cfg['envelopes']}, Size={cfg['size']}")

In [ ]:
# Fonction helper pour run un backtest
def run_single_backtest(df_list, oldest_pair, params_coin, stop_loss, params_adapter=None):
    """
    Exécute un backtest avec les paramètres donnés.
    
    Returns:
        dict: Résultat du backtest (trades, days, wallet, metrics)
    """
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    result = strategy.run_backtest(
        **BACKTEST_PARAMS,
        stop_loss=stop_loss,
        params_adapter=params_adapter
    )
    
    return result

print("✅ Fonction run_single_backtest définie")

In [ ]:
# Calculer les régimes sur TOUTE la période (pour comparaison manuelle seulement)
# ⚠️ Pour Walk-Forward, on recalculera par fold
regime_series_full = calculate_regime_series(df_btc_full, confirm_n=12)

print("📊 Distribution des régimes (2022-2024):")
regime_counts = regime_series_full.value_counts(normalize=True) * 100
for regime, pct in regime_counts.items():
    print(f"   {regime.name}: {pct:.1f}%")

In [ ]:
# Comparaison manuelle : Fixed vs Adaptive pour chaque config
comparator_manual = BacktestComparator(initial_wallet=INITIAL_WALLET)

print("\n🚀 Exécution des backtests manuels (Fixed + Adaptive)...\n")
print("=" * 80)

for config_name, config in tqdm(MANUAL_CONFIGS.items(), desc="Configurations"):
    # Préparer params_coin
    params_coin = {}
    for pair in PAIRS:
        params_coin[pair] = {
            "src": "close",
            "ma_base_window": config["ma_base_window"],
            "envelopes": config["envelopes"],
            "size": config["size"] / BACKTEST_LEVERAGE
        }
    
    # 1. Fixed params
    adapter_fixed = FixedParamsAdapter(params_coin)
    result_fixed = run_single_backtest(
        df_list_full, oldest_pair, params_coin, 
        config["stop_loss"], adapter_fixed
    )
    
    comparator_manual.add_backtest(
        name=f"{config_name} (Fixed)",
        df_trades=result_fixed['trades'],
        df_days=result_fixed['days'],
        metadata={"config": config, "adaptive": False}
    )
    
    # 2. Adaptive params
    adapter_adaptive = RegimeBasedAdapter(
        base_params=params_coin,
        regime_series=regime_series_full,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    )
    result_adaptive = run_single_backtest(
        df_list_full, oldest_pair, params_coin,
        config["stop_loss"], adapter_adaptive
    )
    
    comparator_manual.add_backtest(
        name=f"{config_name} (Adaptive)",
        df_trades=result_adaptive['trades'],
        df_days=result_adaptive['days'],
        metadata={"config": config, "adaptive": True}
    )

print("\n" + "=" * 80)
print("✅ Backtests manuels terminés\n")

In [ ]:
# Afficher les résultats
comparator_manual.print_summary()

In [ ]:
# Sauvegarder les résultats manuels
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
comparator_manual.save_comparison(f"results_manual_{timestamp}.csv")
print(f"💾 Résultats sauvegardés: results_manual_{timestamp}.csv")

## 3️⃣ Walk-Forward Optimization (Grid Search)

Teste toutes les combinaisons de paramètres avec validation robuste.

In [ ]:
# Grid de paramètres (Phase 1 : grossier)
PARAM_GRID = {
    "ma_base_window": [5, 7, 10],
    "envelope_sets": [
        [0.07, 0.10, 0.15],  # Standard
        [0.08, 0.12, 0.16],  # Large
    ],
    "size": [0.08, 0.10],
    "stop_loss": [0.20, 0.25],
}

# Générer toutes les combinaisons
grid_combinations = list(product(
    PARAM_GRID["ma_base_window"],
    PARAM_GRID["envelope_sets"],
    PARAM_GRID["size"],
    PARAM_GRID["stop_loss"]
))

print(f"🔍 Grid Search Phase 1 (Coarse)")
print(f"   Combinaisons: {len(grid_combinations)}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
print(f"   Total backtests: {len(grid_combinations) * len(WF_FOLDS) * 2} (fixed + adaptive)")
print(f"   Temps estimé: ~{len(grid_combinations) * len(WF_FOLDS) * 2 * 10 / 60:.0f} min")

In [ ]:
# Fonction pour filtrer DataFrame par dates
def filter_df_by_dates(df, start_date, end_date):
    """Filtre un DataFrame par dates."""
    mask = (df.index >= pd.Timestamp(start_date)) & (df.index <= pd.Timestamp(end_date))
    return df[mask]

def filter_df_list_by_dates(df_list, start_date, end_date):
    """Filtre un dict de DataFrames par dates."""
    return {pair: filter_df_by_dates(df, start_date, end_date) for pair, df in df_list.items()}

print("✅ Fonctions de filtrage définies")

In [ ]:
# Fonction de calcul du score composite anti-overfitting
def calculate_composite_score(bt_result, train_sharpe=None):
    """
    Calcule le score composite pour évaluer une configuration.
    
    Args:
        bt_result: Résultat du backtest
        train_sharpe: Sharpe du train (pour consistency), None si on calcule train
    
    Returns:
        float: Score composite (plus élevé = meilleur)
    """
    df_trades = bt_result['trades']
    df_days = bt_result['days']
    
    # Calculer métriques de base
    n_trades = len(df_trades)
    
    # Filtre: trop peu de trades = pas fiable
    if n_trades < 30:
        return -999
    
    # Sharpe ratio
    sharpe = bt_result.get('sharpe_ratio', 0)
    if pd.isna(sharpe) or np.isinf(sharpe):
        sharpe = 0
    
    # Max Drawdown
    df_days_copy = df_days.copy()
    df_days_copy['cummax'] = df_days_copy['wallet'].cummax()
    df_days_copy['drawdown_pct'] = (df_days_copy['wallet'] - df_days_copy['cummax']) / df_days_copy['cummax']
    max_dd = abs(df_days_copy['drawdown_pct'].min()) * 100
    
    # Calmar Ratio (return / max_dd)
    final_wallet = df_days['wallet'].iloc[-1]
    total_return = (final_wallet / INITIAL_WALLET - 1) * 100
    calmar = total_return / max(max_dd, 1.0)  # Éviter division par 0
    
    # Win Rate
    df_trades_copy = df_trades.copy()
    if 'trade_result' not in df_trades_copy.columns:
        df_trades_copy['trade_result'] = (
            df_trades_copy["close_trade_size"] -
            df_trades_copy["open_trade_size"] -
            df_trades_copy["open_fee"] -
            df_trades_copy["close_fee"]
        )
    win_rate = (df_trades_copy['trade_result'] > 0).mean()
    
    # Profit Factor
    gross_profit = df_trades_copy[df_trades_copy['trade_result'] > 0]['trade_result'].sum()
    gross_loss = abs(df_trades_copy[df_trades_copy['trade_result'] < 0]['trade_result'].sum())
    profit_factor = gross_profit / max(gross_loss, 1.0) if gross_loss > 0 else gross_profit
    profit_factor_normalized = min(profit_factor / 2, 1)  # Cap à 1
    
    # Consistency (train vs test)
    if train_sharpe is not None:
        consistency = 1 - abs(train_sharpe - sharpe) / max(0.1, abs(train_sharpe))
        consistency = max(0, consistency)  # Clamp à 0
    else:
        consistency = 0  # Pas de consistency pour train
    
    # Score composite
    if train_sharpe is None:  # Train
        score = (
            sharpe * 0.35 +
            calmar * 0.25 +
            (1 - min(max_dd, 100) / 100) * 0.20 +
            win_rate * 0.10 +
            profit_factor_normalized * 0.10
        )
    else:  # Test
        score = (
            sharpe * 0.30 +
            consistency * 0.25 +
            calmar * 0.20 +
            (1 - min(max_dd, 100) / 100) * 0.15 +
            win_rate * 0.05 +
            profit_factor_normalized * 0.05
        )
    
    return score

print("✅ Fonction calculate_composite_score définie")

In [ ]:
# Walk-Forward Optimization Loop
wf_results = []

print("\n🚀 Démarrage Walk-Forward Optimization...\n")
print("=" * 80)

total_iterations = len(WF_FOLDS) * len(grid_combinations) * 2  # × 2 pour fixed + adaptive
pbar = tqdm(total=total_iterations, desc="Walk-Forward Progress")

for fold in WF_FOLDS:
    fold_name = fold["name"]
    print(f"\n📂 {fold_name}")
    print(f"   Train: {fold['train_start']} → {fold['train_end']}")
    print(f"   Test:  {fold['test_start']} → {fold['test_end']}")
    
    # Filtrer données par période
    df_list_train = filter_df_list_by_dates(df_list_full, fold['train_start'], fold['train_end'])
    df_list_test = filter_df_list_by_dates(df_list_full, fold['test_start'], fold['test_end'])
    
    df_btc_train = filter_df_by_dates(df_btc_full, fold['train_start'], fold['train_end'])
    df_btc_test = filter_df_by_dates(df_btc_full, fold['test_start'], fold['test_end'])
    
    # ⚠️ IMPORTANT: Calculer régimes par fold (évite lookahead bias)
    regime_train = calculate_regime_series(df_btc_train, confirm_n=12)
    regime_test = calculate_regime_series(df_btc_test, confirm_n=12)
    
    for combo_idx, (ma_window, envelopes, size, stop_loss) in enumerate(grid_combinations):
        # Préparer params_coin
        params_coin = {}
        for pair in PAIRS:
            params_coin[pair] = {
                "src": "close",
                "ma_base_window": ma_window,
                "envelopes": envelopes,
                "size": size / BACKTEST_LEVERAGE
            }
        
        # === TRAIN ===
        # 1. Fixed
        adapter_fixed = FixedParamsAdapter(params_coin)
        bt_train_fixed = run_single_backtest(
            df_list_train, oldest_pair, params_coin, stop_loss, adapter_fixed
        )
        score_train_fixed = calculate_composite_score(bt_train_fixed)
        pbar.update(1)
        
        # 2. Adaptive
        adapter_adaptive_train = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_train,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
        bt_train_adaptive = run_single_backtest(
            df_list_train, oldest_pair, params_coin, stop_loss, adapter_adaptive_train
        )
        score_train_adaptive = calculate_composite_score(bt_train_adaptive)
        pbar.update(1)
        
        # === TEST ===
        # 1. Fixed
        adapter_fixed_test = FixedParamsAdapter(params_coin)
        bt_test_fixed = run_single_backtest(
            df_list_test, oldest_pair, params_coin, stop_loss, adapter_fixed_test
        )
        sharpe_train_fixed = bt_train_fixed.get('sharpe_ratio', 0)
        score_test_fixed = calculate_composite_score(bt_test_fixed, sharpe_train_fixed)
        
        # 2. Adaptive
        adapter_adaptive_test = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_test,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
        bt_test_adaptive = run_single_backtest(
            df_list_test, oldest_pair, params_coin, stop_loss, adapter_adaptive_test
        )
        sharpe_train_adaptive = bt_train_adaptive.get('sharpe_ratio', 0)
        score_test_adaptive = calculate_composite_score(bt_test_adaptive, sharpe_train_adaptive)
        
        # Stocker résultats
        wf_results.append({
            "fold": fold_name,
            "combo_idx": combo_idx,
            "ma_window": ma_window,
            "envelopes": str(envelopes),
            "size": size,
            "stop_loss": stop_loss,
            "adaptive": False,
            "train_wallet": bt_train_fixed['wallet'],
            "train_sharpe": sharpe_train_fixed,
            "train_score": score_train_fixed,
            "train_trades": len(bt_train_fixed['trades']),
            "test_wallet": bt_test_fixed['wallet'],
            "test_sharpe": bt_test_fixed.get('sharpe_ratio', 0),
            "test_score": score_test_fixed,
            "test_trades": len(bt_test_fixed['trades']),
        })
        
        wf_results.append({
            "fold": fold_name,
            "combo_idx": combo_idx,
            "ma_window": ma_window,
            "envelopes": str(envelopes),
            "size": size,
            "stop_loss": stop_loss,
            "adaptive": True,
            "train_wallet": bt_train_adaptive['wallet'],
            "train_sharpe": sharpe_train_adaptive,
            "train_score": score_train_adaptive,
            "train_trades": len(bt_train_adaptive['trades']),
            "test_wallet": bt_test_adaptive['wallet'],
            "test_sharpe": bt_test_adaptive.get('sharpe_ratio', 0),
            "test_score": score_test_adaptive,
            "test_trades": len(bt_test_adaptive['trades']),
        })

pbar.close()
print("\n" + "=" * 80)
print("✅ Walk-Forward Optimization terminée\n")

In [ ]:
# Créer DataFrame des résultats
df_wf_results = pd.DataFrame(wf_results)

# Calculer moyenne des scores sur les folds
df_wf_avg = df_wf_results.groupby(['combo_idx', 'ma_window', 'envelopes', 'size', 'stop_loss', 'adaptive']).agg({
    'train_score': 'mean',
    'test_score': 'mean',
    'train_sharpe': 'mean',
    'test_sharpe': 'mean',
    'train_trades': 'sum',
    'test_trades': 'sum',
}).reset_index()

# Calculer consistency
df_wf_avg['consistency'] = 1 - abs(df_wf_avg['train_sharpe'] - df_wf_avg['test_sharpe']) / df_wf_avg['train_sharpe'].abs().clip(lower=0.1)
df_wf_avg['consistency'] = df_wf_avg['consistency'].clip(lower=0)

# Score final = moyenne test_score
df_wf_avg = df_wf_avg.sort_values('test_score', ascending=False)

print("✅ Résultats agrégés")
print(f"   Total configurations testées: {len(df_wf_avg)}")
print(f"   Folds par configuration: {len(WF_FOLDS)}")

In [ ]:
# Top 10 configurations
print("\n🏆 TOP 10 CONFIGURATIONS (par Test Score moyen)\n")
print("=" * 120)

top10 = df_wf_avg.head(10)
for idx, row in top10.iterrows():
    print(f"#{idx+1}")
    print(f"   MA: {row['ma_window']}, Env: {row['envelopes']}, Size: {row['size']}, SL: {row['stop_loss']}, Adaptive: {row['adaptive']}")
    print(f"   Train Sharpe: {row['train_sharpe']:.2f}, Test Sharpe: {row['test_sharpe']:.2f}, Consistency: {row['consistency']:.2f}")
    print(f"   Train Score: {row['train_score']:.3f}, Test Score: {row['test_score']:.3f}")
    print(f"   Trades: Train={row['train_trades']}, Test={row['test_trades']}")
    print()

# Identifier le meilleur
best_config = top10.iloc[0]
print("\n" + "=" * 120)
print(f"✅ MEILLEURE CONFIGURATION:")
print(f"   MA: {best_config['ma_window']}")
print(f"   Envelopes: {best_config['envelopes']}")
print(f"   Size: {best_config['size']}")
print(f"   Stop Loss: {best_config['stop_loss']}")
print(f"   Adaptive: {best_config['adaptive']}")
print(f"   Test Score: {best_config['test_score']:.3f}")

## 4️⃣ Validation Hold-out finale

Test **UNE SEULE FOIS** sur les données de hold-out (2024 H2) pour vérifier qu'il n'y a pas d'overfitting.

In [ ]:
print("\n⚠️  VALIDATION HOLD-OUT FINALE")
print("=" * 80)
print(f"Période: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")
print(f"⚠️  Cette validation ne peut être exécutée qu'UNE SEULE FOIS !\n")

# Filtrer données hold-out
df_list_holdout = filter_df_list_by_dates(df_list_full, PERIODS['holdout']['start'], PERIODS['holdout']['end'])
df_btc_holdout = filter_df_by_dates(df_btc_full, PERIODS['holdout']['start'], PERIODS['holdout']['end'])
regime_holdout = calculate_regime_series(df_btc_holdout, confirm_n=12)

# Préparer params de la meilleure config
params_coin_best = {}
for pair in PAIRS:
    params_coin_best[pair] = {
        "src": "close",
        "ma_base_window": int(best_config['ma_window']),
        "envelopes": eval(best_config['envelopes']),  # Convertir string → list
        "size": float(best_config['size']) / BACKTEST_LEVERAGE
    }

# Test hold-out
if best_config['adaptive']:
    adapter_holdout = RegimeBasedAdapter(
        base_params=params_coin_best,
        regime_series=regime_holdout,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    )
else:
    adapter_holdout = FixedParamsAdapter(params_coin_best)

bt_holdout = run_single_backtest(
    df_list_holdout, oldest_pair, params_coin_best,
    float(best_config['stop_loss']), adapter_holdout
)

# Calculer métriques hold-out
holdout_sharpe = bt_holdout.get('sharpe_ratio', 0)
holdout_wallet = bt_holdout['wallet']
holdout_perf = (holdout_wallet / INITIAL_WALLET - 1) * 100
holdout_trades = len(bt_holdout['trades'])

# Comparaison
train_sharpe_avg = best_config['train_sharpe']
test_sharpe_avg = best_config['test_sharpe']

print(f"\n📊 RÉSULTATS HOLD-OUT:")
print(f"   Wallet final: ${holdout_wallet:.2f}")
print(f"   Performance: {holdout_perf:+.2f}%")
print(f"   Sharpe Ratio: {holdout_sharpe:.2f}")
print(f"   Nombre de trades: {holdout_trades}")

print(f"\n📊 COMPARAISON:")
print(f"   Train Sharpe (avg): {train_sharpe_avg:.2f}")
print(f"   Test Sharpe (avg):  {test_sharpe_avg:.2f}")
print(f"   Hold-out Sharpe:    {holdout_sharpe:.2f}")

# Validation
sharpe_diff_train = abs(holdout_sharpe - train_sharpe_avg)
sharpe_diff_test = abs(holdout_sharpe - test_sharpe_avg)

if sharpe_diff_test <= 0.5:
    print(f"\n✅ VALIDATION RÉUSSIE: Hold-out Sharpe ≈ Test Sharpe (diff={sharpe_diff_test:.2f})")
    print(f"   Pas d'overfitting détecté. Configuration robuste.")
elif sharpe_diff_test <= 1.0:
    print(f"\n⚠️  WARNING: Hold-out Sharpe diverge légèrement (diff={sharpe_diff_test:.2f})")
    print(f"   Overfitting possible mais acceptable.")
else:
    print(f"\n❌ ÉCHEC: Hold-out Sharpe diverge fortement (diff={sharpe_diff_test:.2f})")
    print(f"   Overfitting détecté ! Réviser les paramètres.")

print("\n" + "=" * 80)

## 5️⃣ Résultats et visualisation

In [ ]:
# Scatter plot : Train Sharpe vs Test Sharpe
fig, ax = plt.subplots(figsize=(10, 8))

# Séparer Fixed et Adaptive
df_fixed = df_wf_avg[df_wf_avg['adaptive'] == False]
df_adaptive = df_wf_avg[df_wf_avg['adaptive'] == True]

ax.scatter(df_fixed['train_sharpe'], df_fixed['test_sharpe'], 
           alpha=0.6, s=100, label='Fixed', marker='o')
ax.scatter(df_adaptive['train_sharpe'], df_adaptive['test_sharpe'], 
           alpha=0.6, s=100, label='Adaptive', marker='^')

# Ligne y=x (pas d'overfitting)
max_sharpe = max(df_wf_avg['train_sharpe'].max(), df_wf_avg['test_sharpe'].max())
min_sharpe = min(df_wf_avg['train_sharpe'].min(), df_wf_avg['test_sharpe'].min())
ax.plot([min_sharpe, max_sharpe], [min_sharpe, max_sharpe], 
        'r--', alpha=0.5, label='No overfitting (y=x)')

# Zone acceptable (±0.5)
ax.fill_between([min_sharpe, max_sharpe], 
                 [min_sharpe - 0.5, max_sharpe - 0.5],
                 [min_sharpe + 0.5, max_sharpe + 0.5],
                 alpha=0.1, color='green', label='Acceptable zone (±0.5)')

ax.set_xlabel('Train Sharpe Ratio', fontsize=12)
ax.set_ylabel('Test Sharpe Ratio', fontsize=12)
ax.set_title('Train vs Test Sharpe Ratio (Overfitting Detection)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'train_vs_test_sharpe_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: train_vs_test_sharpe_{timestamp}.png")

In [ ]:
# Bar chart : Top 10 configurations par test score
fig, ax = plt.subplots(figsize=(14, 8))

top10_display = df_wf_avg.head(10).copy()
top10_display['config_label'] = (
    'MA=' + top10_display['ma_window'].astype(str) + 
    ', Size=' + top10_display['size'].astype(str) +
    ', ' + top10_display['adaptive'].map({True: 'Adapt', False: 'Fixed'})
)

x = range(len(top10_display))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], top10_display['train_score'], 
               width, label='Train Score', alpha=0.8, color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], top10_display['test_score'], 
               width, label='Test Score', alpha=0.8, color='coral')

ax.set_xlabel('Configuration', fontsize=12)
ax.set_ylabel('Composite Score', fontsize=12)
ax.set_title('Top 10 Configurations by Test Score', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(top10_display['config_label'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'top10_scores_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: top10_scores_{timestamp}.png")

In [ ]:
# Sauvegarder tous les résultats
df_wf_results.to_csv(f"wf_results_detailed_{timestamp}.csv", index=False)
df_wf_avg.to_csv(f"wf_results_summary_{timestamp}.csv", index=False)

# Sauvegarder meilleure config en JSON
import json

best_config_export = {
    "ma_base_window": int(best_config['ma_window']),
    "envelopes": eval(best_config['envelopes']),
    "size": float(best_config['size']),
    "stop_loss": float(best_config['stop_loss']),
    "adaptive": bool(best_config['adaptive']),
    "train_sharpe": float(best_config['train_sharpe']),
    "test_sharpe": float(best_config['test_sharpe']),
    "test_score": float(best_config['test_score']),
    "holdout_sharpe": float(holdout_sharpe),
    "holdout_perf": float(holdout_perf),
    "timestamp": timestamp,
}

with open(f"best_config_{timestamp}.json", 'w') as f:
    json.dump(best_config_export, f, indent=2)

print(f"\n💾 Résultats sauvegardés:")
print(f"   - wf_results_detailed_{timestamp}.csv (tous les backtests)")
print(f"   - wf_results_summary_{timestamp}.csv (moyennes par config)")
print(f"   - best_config_{timestamp}.json (meilleure configuration)")

## 🎯 Recommandation finale

In [ ]:
print("\n" + "=" * 80)
print("🎯 RECOMMANDATION FINALE")
print("=" * 80)

print(f"\n✅ Meilleure configuration identifiée:")
print(f"   ma_base_window: {best_config['ma_window']}")
print(f"   envelopes: {best_config['envelopes']}")
print(f"   size: {best_config['size']}")
print(f"   stop_loss: {best_config['stop_loss']}")
print(f"   adaptive: {best_config['adaptive']}")

print(f"\n📊 Performance validée:")
print(f"   Train Sharpe (avg): {train_sharpe_avg:.2f}")
print(f"   Test Sharpe (avg): {test_sharpe_avg:.2f}")
print(f"   Hold-out Sharpe: {holdout_sharpe:.2f}")
print(f"   Consistency: {best_config['consistency']:.2f}")

if sharpe_diff_test <= 0.5:
    print(f"\n✅ Validation: Configuration robuste (pas d'overfitting)")
    print(f"   → Recommandé pour mise en production")
elif sharpe_diff_test <= 1.0:
    print(f"\n⚠️  Validation: Overfitting léger détecté")
    print(f"   → Utiliser avec prudence, surveiller en live")
else:
    print(f"\n❌ Validation: Overfitting significatif")
    print(f"   → NE PAS utiliser en production")
    print(f"   → Réduire la complexité du grid ou augmenter les données")

print(f"\n📝 Prochaines étapes:")
print(f"   1. Tester la config sur les 28 paires complètes")
print(f"   2. Valider sur forward test (paper trading)")
print(f"   3. Si validé, appliquer dans multi_envelope.ipynb")

print("\n" + "=" * 80)